# Feature Engineering

In [121]:
import pandas as pd
import numpy as np

In [122]:
pd.set_option("display.max_columns", None)

In [123]:
all_games = pd.read_csv("data/games_clean.csv", index_col="id")

In [124]:
all_games.head()

,date,home_team_score,period,postseason,season,status,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,home_team.full_name,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division,visitor_team.full_name
id,,,,,,,,,,,,,,,,,
47179,2019-01-30T00:00:00.000Z,126,4,False,2018,Final,94,2,BOS,East,Atlantic,Boston Celtics,4,CHA,East,Southeast,Charlotte Hornets
48751,2019-02-09T00:00:00.000Z,112,4,False,2018,Final,123,2,BOS,East,Atlantic,Boston Celtics,13,LAC,West,Pacific,LA Clippers
48739,2019-02-08T00:00:00.000Z,117,4,False,2018,Final,110,23,PHI,East,Atlantic,Philadelphia 76ers,8,DEN,West,Northwest,Denver Nuggets
48740,2019-02-08T00:00:00.000Z,119,4,False,2018,Final,106,30,WAS,East,Southeast,Washington Wizards,6,CLE,East,Central,Cleveland Cavaliers
48746,2019-02-08T00:00:00.000Z,102,4,False,2018,Final,96,26,SAC,West,Pacific,Sacramento Kings,16,MIA,East,Southeast,Miami Heat


In [128]:
# Feature engineering

g = all_games.copy()

g["date"] = pd.to_datetime(games["date"]).dt.tz_localize(None)

# creating target variable
g["winner"] = np.where(g["home_team_score"] > g["visitor_team_score"], 1, 0)

# creating avg points by team across all years (not sure how useful this will be)
home_avg_pts_map = g[["home_team.id", "home_team_score"]].groupby("home_team.id").mean().squeeze()
g["home_team_avg_score_historical"] = g["home_team.id"].map(home_avg_pts_map).round(1)

visitor_avg_pts_map = g[["visitor_team.id", "visitor_team_score"]].groupby("visitor_team.id").mean().squeeze()
g["visitor_team_avg_score_historical"] = g["visitor_team.id"].map(visitor_avg_pts_map).round(1)

In [205]:
# creating an identifier for the team in a specific season
g["home_team_id_year"] = (g["home_team.id"].astype(str) + " " + g["season"].astype(str)).values
g["visitor_team_id_year"] = (g["visitor_team.id"].astype(str) + " " + g["season"].astype(str)).values

In [206]:
# creating average pts feature
# average pts scored per game in that season
# based on if they are home or away because there is a statistically significant difference
home_avg_score_map = g[["home_team_id_year", "home_team_score"]].groupby(["home_team_id_year"]).mean().squeeze()
visitor_avg_score_map = g[["visitor_team_id_year", "visitor_team_score"]].groupby(["visitor_team_id_year"]).mean().squeeze()

g["home_team_avg_score"] = g["home_team_id_year"].map(home_avg_score_map).round(1)
g["visitor_team_avg_score"] = g["visitor_team_id_year"].map(visitor_avg_score_map).round(1) 

In [220]:
# create an average difference in pts feature
# will calculate on average how much a team wins or loses by
# this will hopefully benefit lower scoring teams that also play defensively
avg_score_diff = g[["home_team_id_year", "home_team_avg_score", "visitor_team_avg_score"]].groupby("home_team_id_year").mean()
avg_score_diff = avg_score_diff["home_team_avg_score"] - avg_score_diff["visitor_team_avg_score"]
g["home_avg_score_diff"] = g["home_team_id_year"].map(avg_score_diff)

avg_score_diff = g[["visitor_team_id_year", "visitor_team_avg_score", "home_team_avg_score"]].groupby("visitor_team_id_year").mean()
avg_score_diff = avg_score_diff["visitor_team_avg_score"] - avg_score_diff["home_team_avg_score"]
g["visitor_avg_score_diff"] = g["visitor_team_id_year"].map(avg_score_diff)

In [221]:
g

,date,home_team_score,period,postseason,season,status,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,home_team.full_name,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division,visitor_team.full_name,winner,home_team_avg_score_historical,visitor_team_avg_score_historical,home_team_id_year,visitor_team_id_year,home_team_avg_score,visitor_team_avg_score,home_avg_score_diff,visitor_avg_score_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,
47179,2019-01-30,126,4,False,2018,Final,94,2,BOS,East,Atlantic,Boston Celtics,4,CHA,East,Southeast,Charlotte Hornets,1,105.8,98.2,2 2018,4 2018,112.8,108.3,3.620000,-3.831707
48751,2019-02-09,112,4,False,2018,Final,123,2,BOS,East,Atlantic,Boston Celtics,13,LAC,West,Pacific,LA Clippers,0,105.8,100.7,2 2018,13 2018,112.8,113.1,3.620000,0.581818
48739,2019-02-08,117,4,False,2018,Final,110,23,PHI,East,Atlantic,Philadelphia 76ers,8,DEN,West,Northwest,Denver Nuggets,1,103.3,104.2,23 2018,8 2018,117.9,108.2,8.725532,-4.670213
48740,2019-02-08,119,4,False,2018,Final,106,30,WAS,East,Southeast,Washington Wizards,6,CLE,East,Central,Cleveland Cavaliers,1,103.4,98.3,30 2018,6 2018,116.4,103.8,7.429268,-8.419512
48746,2019-02-08,102,4,False,2018,Final,96,26,SAC,West,Pacific,Sacramento Kings,16,MIA,East,Southeast,Miami Heat,1,105.5,96.0,26 2018,16 2018,114.9,105.4,5.129268,-6.670732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128052,2021-03-03,111,4,False,2020,Final,114,6,CLE,East,Central,Cleveland Cavaliers,12,IND,East,Central,Indiana Pacers,0,102.3,100.1,6 2020,12 2020,107.5,117.7,-4.341667,5.432432
128036,2021-03-01,124,4,False,2020,Final,130,22,ORL,East,Southeast,Orlando Magic,7,DAL,West,Southwest,Dallas Mavericks,0,101.9,101.5,22 2020,7 2020,105.4,113.8,-5.902778,1.257500
128001,2021-02-25,111,4,False,2020,Final,97,23,PHI,East,Atlantic,Philadelphia 76ers,7,DAL,West,Southwest,Dallas Mavericks,1,103.3,101.5,23 2020,7 2020,117.3,113.8,6.072093,1.257500


In [222]:
g.to_csv("data/games_with_features.csv")

In [86]:
feat_names = ["season", "winner", "home_team_avg_score", "visitor_team_avg_score"]
features = g.loc[:, feat_names]

features.to_csv("data/features.csv")

### Using stats

In [313]:
stats = pd.read_csv("data/stats2020_clean.csv", index_col="id")

In [315]:
stats["game_date"] = pd.to_datetime(stats["game_date"]).dt.tz_localize(None)

In [317]:
stats["min"] = [pd.Timedelta(minutes=int(time[0]), seconds=int(time[1])) for time in stats["min"].str.split(":").values]

In [319]:
# split into 2 dataframes so that grouping data by game id doesn't
# group players of opposing teams
home_stats = stats[stats["player_team_id"].eq(stats["home_team_id"])]
away_stats = stats[stats["player_team_id"].eq(stats["away_team_id"])]

In [332]:
agg_map = {"ast": "sum", 
           "blk": "sum", 
           "dreb": "sum", 
           "fg3_pct": "mean", 
           "fg3a": "sum", 
           "fg3m": "sum", 
           "fg_pct": "mean",
          "fga": "sum",
          "fgm": "sum",
          "ft_pct": "mean",
          "fta": "sum",
          "ftm": "sum",
          "min": "sum",
          "oreb": "sum",
          "pf": "sum",
          "pts": "sum",
          "reb": "sum",
          "stl": "sum",
          "turnover": "sum",
          "player_id": "first",
          "player_team_id": "first",
          "game_id": "first",
          "game_date": "first",
          "season": "first",
          "home_team_id": "first",
          "home_team_score": "first",
          "away_team_id": "first",
          "away_team_score": "first"}

In [335]:
# aggregate player stats to team stats
home_games = home_stats.groupby("game_id").agg(agg_map)
away_games = away_stats.groupby("game_id").agg(agg_map)

# player stats were aggregated to team stats, so player id doesn't make sense anymore
home_games.drop(["player_id", "player_team_id"], axis=1, inplace=True)
away_games.drop(["player_id", "player_team_id"], axis=1, inplace=True)

In [341]:
home_games.sort_values(["home_team_id", "game_date"])

,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game_id,game_date,season,home_team_id,home_team_score,away_team_id,away_team_score
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,
127542,30,7,37,42.430000,42,20,53.720000,82,40,50.000000,32,28,0 days 03:59:58,8,22,128,45,5,13,127542,2020-12-28,2020,1,128,9,120
127581,22,5,33,20.330000,32,11,35.850000,85,35,20.830000,19,10,0 days 03:59:59,15,20,91,48,9,20,127581,2021-01-02,2020,1,91,6,96
127596,24,2,30,22.222222,31,10,36.511111,87,40,38.522222,23,18,0 days 04:00:00,10,21,108,40,4,12,127596,2021-01-04,2020,1,108,20,113
127611,25,1,45,7.600000,40,7,33.260000,96,36,42.140000,18,15,0 days 03:59:57,12,25,94,57,6,13,127611,2021-01-06,2020,1,94,4,102
127655,20,3,44,26.591667,44,17,31.425000,99,37,44.641667,27,21,0 days 04:00:00,14,13,112,58,7,8,127655,2021-01-11,2020,1,112,23,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264689,50,7,35,32.400000,22,9,61.263636,103,63,58.181818,24,19,0 days 03:59:58,9,19,154,44,10,16,264689,2021-05-03,2020,30,154,12,141
264801,22,3,43,14.718182,31,10,33.936364,91,44,23.700000,21,17,0 days 03:59:57,10,20,115,53,4,14,264801,2021-05-16,2020,30,115,4,110
423334,34,9,45,31.457143,28,14,60.114286,93,54,45.835714,26,20,0 days 03:59:59,7,20,142,52,7,15,423334,2021-05-20,2020,30,142,12,115
